Comando para abrir el server desde la consaola:

jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0 --no-browser --NotebookApp.token=abcd

Luego copiar token de la consola al google collab para conectarse de forma local.

Se le puede agregar el siguiente parametro para tener un token fijo.

--NotebookApp.token=abcd

In [19]:
#!pip install iar
#!pip install tensorflow

In [20]:
import iar
from iar import IAR_gamma
import numpy as np
import matplotlib.pyplot as plt
from os import listdir, path, mkdir
import os.path
from os.path import isfile, join
import shutil
from configparser import ConfigParser
from datetime import datetime
import random
import keras
import pandas as pd
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler

print("iAR version:") #1.0.2 o 1.0.3
#A la fecha 30/01/2022, el numero de version de la libreria de iAR no se actualiza. Esta version corresponde a la 1.0.2 o 1.0.3, no me acuerdo bien.
print(iar.__version__)

iAR version:
1.0.0


In [21]:
#ruta_datos = '.\\Archivos_profe\\DATOSWISE\\LCs' #RUTA DE LAS SERIES DE TIEMPO DEL DATASET
#path_index = '.\\Archivos_profe\\DATOSWISE\\WISE_dataset.dat' #RUTA DEL INDEX DEL DATASET
#ruta_instancias = '.\\Archivos_profe\\modelos\\Experimento_2_capa\\WISE' #INSTANCIAS QUE GENERO DEL CODIGO DE LA GAN, FOLDEOS, TRAINING, PLOTS, ETC.

#ruta_datos = '.\\Archivos_profe\\DATOSGAIA\\LCs' #RUTA DE LAS SERIES DE TIEMPO DEL DATASET
#path_index = '.\\Archivos_profe\\DATOSGAIA\\GAIA_dataset.dat' #RUTA DEL INDEX DEL DATASET
#ruta_instancias = '.\\Archivos_profe\\modelos\\Experimento_2_capa\\GAIA' #INSTANCIAS QUE GENERO DEL CODIGO DE LA GAN, FOLDEOS, TRAINING, PLOTS, ETC.

ruta_datos = '.\\Archivos_profe\\DATOSOGLE\\LCs' #RUTA DE LAS SERIES DE TIEMPO DEL DATASET
path_index = '.\\Archivos_profe\\DATOSOGLE\\OGLE_dataset.dat' #RUTA DEL INDEX DEL DATASET
#ruta_instancias = '.\\Archivos_profe\\modelos\\Experimento_1_capa\\OGLE' #INSTANCIAS QUE GENERO DEL CODIGO DE LA GAN, FOLDEOS, TRAINING, PLOTS, ETC.
ruta_instancias = '.\\Archivos_profe\\modelos' #INSTANCIAS QUE GENERO DEL CODIGO DE LA GAN, FOLDEOS, TRAINING, PLOTS, ETC.

forzar_generacion_de_series = False #True si queremos obligar a generar series de tiempo con promedio de magnitud cercano a las series reales. False, si queremos generar una cantidad de series X sin importar si son buenas o malas.

In [22]:
#Lista de nombre de TODOS los archivos (Si hay más archivos como un txt tambien los va a guardar). HAY QUE ASEGURARSE QUE SOLO HAYA FOTOMETRIAS AQUI.
lista_fotometrias = [f for f in listdir(ruta_datos) if isfile(join(ruta_datos, f))]
print(len(lista_fotometrias)) #EN MI CASO OGLE-III SON 393.103
#Lista de nombre de TODOS los archivos de instancias del experimento
lista_instancias = [f for f in listdir(ruta_instancias)]

393103


In [23]:
lista_accuracy_por_instancia = []
#Por cada instancia vamos a leer los parametros guardados

for nombre_instancia in lista_instancias:
    clear_output()
    lista_fotometrias_IAR = [] #Por cada instancia de la GAN debo guardar x cantidad de series de tiempo generadas por IAR.

    print('Leyendo instancia:', nombre_instancia)
    ruta_modelos = ruta_instancias + '\\' + nombre_instancia + '\\modelos' #Carpeta generada al entrenar la GAN
    ruta_parametro = ruta_instancias + '\\' + nombre_instancia + '\\parametros.init' #Nombre de archivo donde se guardan los parametros de la GAN
    ruta_resultados_IAR = ruta_instancias + '\\' + nombre_instancia + '\\resultados_IAR' #Carpeta donde se almacenan los plots de IAR
    ruta_IAR_entrenado = ruta_resultados_IAR + '\\plot_entrenado'
    ruta_IAR_generado = ruta_resultados_IAR + '\\plot_generado'

    #Si existe una carpeta con ese nombre la borramos.
    if path.exists(ruta_resultados_IAR):
        shutil.rmtree(ruta_resultados_IAR)
    #Creamos las carpetas necesarias.
    mkdir(ruta_resultados_IAR)
    mkdir(ruta_IAR_entrenado)
    mkdir(ruta_IAR_generado)

    parser = ConfigParser()
    parser.read(ruta_parametro)

    seccion = 'Parametros compartidos'
    cantidad_series_de_tiempo = int(parser.get(seccion, 'cantidad_series_de_tiempo')) #nombre_seccion, variable1
    cantidad_mediciones = int(parser.get(seccion, 'cantidad_mediciones')) #nombre_seccion, variable1
    nombre_clase = parser.get(seccion, 'nombre_clase') #nombre_seccion, variable1

    seccion = 'GAN'
    n_input = parser.get(seccion, 'n_input')

    seccion = 'LSTM_score'
    n_samples_evaluate = int(parser.get(seccion, 'n_samples_evaluate')) #nombre_seccion, variable1

    seccion = 'Resultados'
    accuracy_lstm_score_gan = float(parser.get(seccion, 'accuracy_lstm_score_gan'))/100 #nombre_seccion, variable1

    print('cantidad_series_de_tiempo', cantidad_series_de_tiempo)
    print('cantidad_mediciones', cantidad_mediciones)
    print('n_samples_evaluate', n_samples_evaluate)
    print('accuracy_lstm_score_gan', accuracy_lstm_score_gan)

    #Cargamos el modelo LSTM_score
    lstm_score_model = keras.models.load_model(ruta_modelos + "\\lstm_score.h5")
    #lstm_score_model = keras.models.load_model(ruta_modelos + "\\lstm_score")
    
    #FILTRAMOS LAS SERIES DE TIEMPO SEGUN LA CLASE QUE ESTAMOS LEYENDO DE LA INSTANCIA
    columnas_index = ['ID', 'Class', 'Path', 'N']
    #Leemos el archivo de indice o de info de las series de tiempo.
    df_index_original = pd.read_csv(path_index, sep=',', skiprows=1, names=columnas_index)
    #Eliminamos las columnas que no nos interesan.
    df_index_filtrada = df_index_original.drop(columns=['Path'])
    #Filtramos el df solo con la clase que estamos analizando.
    df_index_filtrada = df_index_filtrada.loc[df_index_filtrada['Class'] == nombre_clase]
    #Filtramos el df dejando {cantidad_mediciones} mediciones o más.
    df_index_filtrada = df_index_filtrada.loc[df_index_filtrada['N'] >= cantidad_mediciones]
    if df_index_filtrada.empty:
        print("\n##### EL DF FILTRADO DE INDEX DE FOTOMETRIAS QUEDO VACIO. HELP!! #####\n")
        break

    #Pasamos la lista COMPLETA (incluye todas las clases) a un df.
    df_listdir = pd.DataFrame(lista_fotometrias, columns =['ID'])
    #Para asegurar una union correcta, convertimos las columnas de 'ID' a str. Tambien se puede pasar a int o int64 pero arroja errores con ciertos numeros en particular.
    df_listdir['ID'] = df_listdir['ID'].astype(str)
    df_index_filtrada['ID'] = df_index_filtrada['ID'].astype(str)
    #Realizamos una union entre los df para descartar las series de tiempo que no pertenezcan a la clase.
    df_union = df_index_filtrada.join(df_listdir.set_index('ID'), on='ID')
    #Shuffle al df.
    df_union = df_union.sample(frac=1)
    #Si el .join no funciona se utiliza otro metodo. (para el caso de WISE)
    if df_union.empty:
        df_union = pd.merge(df_index_filtrada, df_listdir, on='ID')
    if df_union.empty:
        print("\n##### EL DF FILTRADO DE FOTOMETRIAS QUEDO VACIO. HELP!! #####\n")
        break

    #Obtenemos una muestra de las series de tiempo reales para pasarlas a la LSTM. (Finalmente dejamos que el tamaño de la muestra fuera igual al total de series).
    lista_fotometrias_reales = []

    for index, row in df_union.iterrows():
        nombre = row['ID'] + '.dat'
        #print('Leyendo fotometria para LSTM:', nombre)
        ruta_fotometria = ruta_datos + '\\' + nombre

        #Leemos la fotometria para pasarla a la LSTM.
        if 'OGLE' in ruta_datos:
            data = np.genfromtxt(ruta_fotometria, unpack=True, delimiter=' ')
        elif 'GAIA' in ruta_datos or 'WISE' in ruta_datos:
            data = np.genfromtxt(ruta_fotometria, unpack=True, delimiter=',', skip_header=1)

        tiempo = data[0,]
        magnitud = data[1,]

        series_de_tiempo_real = np.vstack((tiempo, magnitud)).T
        #print(series_de_tiempo_real.shape)
        series_de_tiempo_real = series_de_tiempo_real[:cantidad_mediciones,:]
        series_de_tiempo_real = np.reshape(series_de_tiempo_real, (cantidad_mediciones, 2))
        #print(series_de_tiempo_real[:10,:])
        lista_fotometrias_reales.append(series_de_tiempo_real)
        #print(series_de_tiempo_real[:10,:])


    #Array de 3D (2000, 700, 2) por ejemplo. Esta se va a usar para la LSTM
    array_fotometrias_reales = np.array(lista_fotometrias_reales, dtype=object, ndmin=3)

    #ITERAMOS POR CADA SERIE DE TIEMPO PREVIAMENTE FILTRADA POR CLASE, CANTIDAD DE SERIES DE TIEMPO Y DE MEDICIONES PARA LUEGO PASARLA A LA LSTM
    numero_serie = 0
    contadorDeZeros = 0
    iteraciones = 0

    for index, row in df_union.iterrows():
        if len(lista_fotometrias_IAR) >= cantidad_series_de_tiempo:
            break
        iteraciones = iteraciones + 1
        nombre = row['ID'] + '.dat'
        print('iteraciones:', iteraciones)
        print('numero_serie:', numero_serie)
        print('Leyendo instancia:', nombre_instancia)
        print('Leyendo fotometria para IAR:', nombre)
        ruta_fotometria = ruta_datos + '\\' + nombre

        #Fotometria input al entrenamiento de IAR
        if 'OGLE' in ruta_datos:
            data = np.genfromtxt(ruta_fotometria, unpack = True, delimiter=' ')
        elif 'GAIA' in ruta_datos or 'WISE' in ruta_datos:
            data = np.genfromtxt(ruta_fotometria, unpack = True, delimiter=',', skip_header=1)

        #Leemos las variables de las series de tiempo
        tiempo = data[0,]
        magnitud = data[1,]

        #Entrenamos modelo IAR
        #print('mean:', np.mean(magnitud))
        #print('std:', np.std(magnitud))

        ruido = np.random.normal(0, 1, magnitud.shape[0])*0.1
        magnitud = magnitud + ruido
        #tiempo = tiempo + ruidoTiempo

        #print('mean:', np.mean(magnitud))
        #print('std:', np.std(magnitud))
        #print(magnitud)

        phi, mu, sigma, loglik = IAR_gamma(magnitud, tiempo)
        print("Estimated parameter:",np.round(phi,4))
        print("Estimated mean:",np.round(mu,4))
        print("Estimated standard deviation:",np.round(sigma,4))

        #print("phi:", phi)
        #print("mu:", mu)
        #print("sigma:", sigma)
        #print("loglik:", loglik)

        #Saltamos la serie generada por iAR que resulta mala
        if forzar_generacion_de_series:
            if mu < 1:
                contadorDeZeros = contadorDeZeros + 1
                print('contadorDeZeros:', contadorDeZeros)
                continue

        if phi == 0 and mu == 0 and sigma == 0:
            print(ruta_fotometria + ' Tiene valores de tiempos repetidos, saltando a la siguiente serie de tiempo')
            continue

        #Ploteamos y guardamos los datos reales
        plt.plot(tiempo, magnitud)
        #plt.clf() #Vacia los datos del plot pero mantiene los ejes.
        plt.xlabel('Tiempo real')
        plt.ylabel('Magnitud real')
        titulo_plot = 'Serie de tiempo - %s' % (row['ID'])
        plt.title(titulo_plot)
        ruta = '%s\\%s-Plot-%s.pdf' % (ruta_IAR_entrenado, numero_serie, row['ID'])
        plt.savefig(ruta, dpi=1200, format='pdf')
        #plt.show()
        plt.clf()
        plt.pause(0.0001)

        #Generamos una serie de tiempo con IAR
        magnitud_IAR, tiempo_IAR = iar.IARg_sample(phi, tiempo.shape[0], tiempo, sigma, mu)
        #Dimensiones de la serie de tiempo generada
        print('magnitud_IAR', magnitud_IAR.shape)
        print('tiempo_IAR', tiempo_IAR.shape)

        #Ploteamos y guardamos la serie de IAR 
        plt.plot(tiempo_IAR[1:,], magnitud_IAR[1:,])
        #plt.clf() #Vacia los datos del plot pero mantiene los ejes.
        plt.xlabel('Tiempo IAR')
        plt.ylabel('Magnitud IAR')
        plt.title('Serie de tiempo generada por IAR')
        ruta = '%s\\%s-Plot-Generado.pdf' % (ruta_IAR_generado, numero_serie)
        plt.savefig(ruta, dpi=1200, format='pdf')
        #plt.show()
        plt.clf()
        plt.pause(0.0001)

        #Procesamos los datos reales de training de IAR para poder guardarlos
        fotometria_real = np.vstack((tiempo, magnitud)).T
        fotometria_real = fotometria_real[:cantidad_mediciones,:]
        fotometria_real = np.reshape(fotometria_real, (cantidad_mediciones, 2))
        #Procesamos los datos generados de IAR para poder guardarlos

        #Work around para evitar datos erroneos que genera el primer dato de la serie de tiempo de IAR.
        tiempo_IAR[0] = tiempo_IAR[1]
        magnitud_IAR[0] = magnitud_IAR[1]

        fotometria_generada = np.vstack((tiempo_IAR, magnitud_IAR)).T
        fotometria_generada = fotometria_generada[:cantidad_mediciones,:]
        fotometria_generada = np.reshape(fotometria_generada, (cantidad_mediciones, 2))
        lista_fotometrias_IAR.append(fotometria_generada)
        print('fotometria_generada', fotometria_generada.shape)
        print('lista_fotometrias_IAR', len(lista_fotometrias_IAR))
        print('\n')

        #GUARDAMOS DATOS USADOS PARA ENTRENAR IAR
        #Pasamos la serie de tiempo a un df
        df_real_IAR = pd.DataFrame(fotometria_real, columns=['time', 'magnitud'])
        #Guardamos los datos usados de training
        ruta = '%s\\%s-SerieTiempo-%s.csv' % (ruta_IAR_entrenado, numero_serie, row['ID'])
        df_real_IAR.to_csv(path_or_buf=ruta, sep=' ', index=False)

        #GUARDAMOS DATOS GENERADOS POR IAR
        #Pasamos la serie de tiempo a un df
        df_generado_IAR = pd.DataFrame(fotometria_generada, columns=['time', 'magnitud'])
        #Guardamos los datos generados de esta evaluacion
        ruta = '%s\\%s-%s' % (ruta_IAR_generado, numero_serie, 'SerieTiempo-Generada.csv')
        df_generado_IAR.to_csv(path_or_buf=ruta, sep=' ', index=False)

        numero_serie = numero_serie + 1
        #Termino de leer las {cantidad_series_de_tiempo} fotometrias

    #Fotometrias generadas por IAR para la LSTM. Tiene que tener las mismas dimensiones que {array_fotometrias_reales}
    array_fotometrias_IAR = np.array(lista_fotometrias_IAR, dtype=object, ndmin=3)
    #Preparamos los datos reales y generados para usar la LSTM
    array_fotometrias_reales = array_fotometrias_reales[:n_samples_evaluate,:,:]
    #print(array_fotometrias_reales[0,:,:])
    array_fotometrias_IAR = array_fotometrias_IAR[:n_samples_evaluate,:,:]

    #TENEMOS QUE PROCESAR LOS DATOS.
    #PARA SERIES REALES Y GENERADAS POR IAR HACER:
    #TIEMPO: DIFERENCIA, NORMALIZAR, SUMA ACUMULADA
    #MAGNITUD: NORMALIZAR
    #Procesamos la magnitud de iar
    magnitud_iar_temporal_3dim = np.delete(array_fotometrias_IAR, (0), axis=2) #Eliminamos el tiempo
    #Cambiamos dimension
    magnitud_iar_temporal_2dim = magnitud_iar_temporal_3dim.reshape([n_samples_evaluate, cantidad_mediciones])
    #Normalizamos
    scaler_magnitud_iar = MinMaxScaler(feature_range=(0, 1))
    scaler_magnitud_iar = scaler_magnitud_iar.fit(magnitud_iar_temporal_2dim.T)
    magnitud_iar_temporal_normalizado_2dim = scaler_magnitud_iar.transform(magnitud_iar_temporal_2dim.T).T
    #Cambiamos dimension
    magnitud_iar_normalizado_3dim = np.reshape(magnitud_iar_temporal_normalizado_2dim, (n_samples_evaluate, cantidad_mediciones, 1))

    #Procesamos el tiempo de iar
    tiempo_iar_temporal_3dim = np.delete(array_fotometrias_IAR, (1), axis=2) #Eliminamos la magnitud
    tiempo_iar_temporal_2dim = tiempo_iar_temporal_3dim.reshape([n_samples_evaluate, cantidad_mediciones])
    #Diferencia de tiempo
    diff_iar_tiempo_sin_norm_2dim_temporal = np.diff(tiempo_iar_temporal_2dim)
    diff_iar_tiempo_sin_norm_2dim = np.zeros_like(tiempo_iar_temporal_2dim)
    diff_iar_tiempo_sin_norm_2dim[:,1:] = diff_iar_tiempo_sin_norm_2dim_temporal
    #Normalizamos tiempo
    scaler_tiempo_iar = MinMaxScaler(feature_range=(0, 1))
    scaler_tiempo_iar = scaler_tiempo_iar.fit(diff_iar_tiempo_sin_norm_2dim.T)
    diff_iar_tiempo_norm_2dim = scaler_tiempo_iar.transform(diff_iar_tiempo_sin_norm_2dim.T).T
    #Cambiamos dimension
    diff_iar_tiempo_norm_3dim = np.reshape(diff_iar_tiempo_norm_2dim,(n_samples_evaluate, cantidad_mediciones, 1))
    #Suma acumulada
    sum_acum_diff_iar_tiempo_norm_3dim = np.cumsum(diff_iar_tiempo_norm_3dim, axis=1)

    #Convertimos el tiempo y magnitud de iar a la variable final que va a la LSTM de score
    array_fotometrias_IAR = np.vstack((sum_acum_diff_iar_tiempo_norm_3dim.T, magnitud_iar_normalizado_3dim.T)).T


    #Procesamos los datos reales
    #Procesamos la magnitud
    magnitud_reaL_temporal_3dim = np.delete(array_fotometrias_reales, (0), axis=2) #Eliminamos el tiempo
    #Cambiamos dimension
    magnitud_reaL_temporal_2dim = magnitud_reaL_temporal_3dim.reshape([n_samples_evaluate, cantidad_mediciones])
    #Normalizamos
    scaler_magnitud_real = MinMaxScaler(feature_range=(0, 1))
    scaler_magnitud_real = scaler_magnitud_real.fit(magnitud_reaL_temporal_2dim.T)
    magnitud_real_temporal_normalizado_2dim = scaler_magnitud_real.transform(magnitud_reaL_temporal_2dim.T).T
    #Cambiamos dimension
    magnitud_real_normalizado_3dim = np.reshape(magnitud_real_temporal_normalizado_2dim, (n_samples_evaluate, cantidad_mediciones, 1))

    #Procesamos el tiempo
    tiempo_real_temporal_3dim = np.delete(array_fotometrias_reales, (1), axis=2) #Eliminamos la magnitud
    tiempo_real_temporal_2dim = tiempo_real_temporal_3dim.reshape([n_samples_evaluate, cantidad_mediciones])
    #Diferencia de tiempo
    diff_real_tiempo_sin_norm_2dim_temporal = np.diff(tiempo_real_temporal_2dim)
    diff_real_tiempo_sin_norm_2dim = np.zeros_like(tiempo_real_temporal_2dim)
    diff_real_tiempo_sin_norm_2dim[:,1:] = diff_real_tiempo_sin_norm_2dim_temporal
    #Normalizamos tiempo
    scaler_tiempo_real = MinMaxScaler(feature_range=(0, 1))
    scaler_tiempo_real = scaler_tiempo_real.fit(diff_real_tiempo_sin_norm_2dim.T)
    diff_real_tiempo_norm_2dim = scaler_tiempo_real.transform(diff_real_tiempo_sin_norm_2dim.T).T
    #Cambiamos dimension
    diff_real_tiempo_norm_3dim = np.reshape(diff_real_tiempo_norm_2dim,(n_samples_evaluate, cantidad_mediciones, 1))
    #Suma acumulada
    sum_acum_diff_real_tiempo_norm_3dim = np.cumsum(diff_real_tiempo_norm_3dim, axis=1)

    #Convertimos el tiempo y magnitud de las series reales a la variable final que va a la LSTM de score
    array_fotometrias_reales = np.vstack((sum_acum_diff_real_tiempo_norm_3dim.T, magnitud_real_normalizado_3dim.T)).T

    #Juntamos las series de tiempo reales con las falsas
    print('array_fotometrias_reales', array_fotometrias_reales.shape)
    print('array_fotometrias_IAR', array_fotometrias_IAR.shape)

    array_fotometrias_reales_generadas = np.concatenate((array_fotometrias_reales[:,:,:], array_fotometrias_IAR[:,:,:]), axis=0)
    #print(array_fotometrias_reales_generadas)

    #Creamos vectores binarios
    vector_binario_1 = np.ones((len(array_fotometrias_reales), 1)) #REAL
    vector_binario_0 = np.zeros((len(array_fotometrias_IAR), 1)) #FALSO
    #Juntamos los vectores binarios
    vectores_binarios = np.concatenate((vector_binario_1, vector_binario_0), axis=0) #Real/Falsa

    #Dividimos los datos. No vamos a utilizar los datos para el entrenamiento ya que la LSTM ya esta entrenada desde la GAN.
    _, x_test, _, y_test = train_test_split(array_fotometrias_reales_generadas, vectores_binarios, test_size=0.2, random_state=5)

    x_test = np.asarray(x_test).astype('float32')
    y_test = np.asarray(y_test).astype('float32')
    
    #INFO
    print('\n')
    print('n_samples_evaluate', n_samples_evaluate) #Reales + Falsas = {cantidad_series_de_tiempo}
    print('vector_binario_1:', vector_binario_1.shape) #Reales
    print('vector_binario_0:', vector_binario_0.shape) #Falsas
    print('vectores_binarios:', vectores_binarios.shape) #Reales + Falsas = {cantidad_series_de_tiempo}
    print('array_fotometrias_reales', array_fotometrias_reales.shape) #Reales
    print('array_fotometrias_IAR', array_fotometrias_IAR.shape) #Falsas
    print('array_fotometrias_reales_generadas:', array_fotometrias_reales_generadas.shape) #Reales + Falsas = {cantidad_series_de_tiempo}
    print("x_test:", x_test.shape) #20% de {cantidad_series_de_tiempo}
    print("y_test", y_test.shape) #20% de {cantidad_series_de_tiempo}


    #print('x_test mag:', x_test[0,0,1])
    #print('x_test type:', type(x_test[0,0,1]))
    #[print(i.shape, i.dtype) for i in lstm_score_model.inputs]
    #[print(o.shape, o.dtype) for o in lstm_score_model.outputs]
    #[print(l.name, l.input_shape, l.dtype) for l in lstm_score_model.layers]

    #Evaluacion de los datos.
    scores = lstm_score_model.evaluate(x_test, y_test, batch_size=None, verbose=1)
    #print(scores)
    print("Accuracy:", scores[1])
    #print("Loss:", scores[0]) #Para este caso no nos importa
    lista_accuracy_por_instancia.append(scores[1])

    #Guardamos los resultados a un archivo de texto
    parser_escritura = ConfigParser()
    seccion_resultados = 'Resultados'
    if not parser_escritura.has_section(seccion_resultados):
        parser_escritura.add_section(seccion_resultados)
    parser_escritura.set(seccion_resultados, 'Accuracy_IAR', str(scores[1]))
    parser_escritura.set(seccion_resultados, 'Accuracy_LSTM_score_GAN', str(accuracy_lstm_score_gan))

    #Guardamos las secciones al archivo.
    resultados_IAR = ruta_resultados_IAR + '\\resultados.txt'
    with open(resultados_IAR, 'a+') as f:
        parser_escritura.write(f)

    print("Resultados guardados en:", resultados_IAR)
    print('contadorDeZeros:', contadorDeZeros) #Cantidad de series generadas con un promedio de magnitud menor a 1.
    print('Voy a leer la siguiente instancia de la GAN\n')

Leyendo instancia: OGLE-30_01_2022-19_23_29-ED
cantidad_series_de_tiempo 2000
cantidad_mediciones 700
n_samples_evaluate 1000
accuracy_lstm_score_gan 0.005179356932640076
iteraciones: 1
numero_serie: 0
Leyendo instancia: OGLE-30_01_2022-19_23_29-ED
Leyendo fotometria para IAR: OGLE-SMC-ECL-1751.dat
Estimated parameter: 0.0
Estimated mean: 18.5819
Estimated standard deviation: 0.0166


<Figure size 432x288 with 0 Axes>

magnitud_IAR (705,)
tiempo_IAR (705,)


<Figure size 432x288 with 0 Axes>

fotometria_generada (700, 2)
lista_fotometrias_IAR 1


iteraciones: 2
numero_serie: 1
Leyendo instancia: OGLE-30_01_2022-19_23_29-ED
Leyendo fotometria para IAR: OGLE-SMC-ECL-1309.dat
Estimated parameter: 0.0
Estimated mean: 19.214
Estimated standard deviation: 0.0236


...

fotometria_generada (700, 2)
lista_fotometrias_IAR 1999


iteraciones: 2140
numero_serie: 1999
Leyendo instancia: OGLE-30_01_2022-19_23_29-ED
Leyendo fotometria para IAR: OGLE-SMC-ECL-0946.dat
Estimated parameter: 0.0
Estimated mean: 16.0112
Estimated standard deviation: 0.0347


<Figure size 432x288 with 0 Axes>

magnitud_IAR (714,)
tiempo_IAR (714,)


<Figure size 432x288 with 0 Axes>

fotometria_generada (700, 2)
lista_fotometrias_IAR 2000


array_fotometrias_reales (1000, 700, 2)
array_fotometrias_IAR (1000, 700, 2)


n_samples_evaluate 1000
vector_binario_1: (1000, 1)
vector_binario_0: (1000, 1)
vectores_binarios: (2000, 1)
array_fotometrias_reales (1000, 700, 2)
array_fotometrias_IAR (1000, 700, 2)
array_fotometrias_reales_generadas: (2000, 700, 2)
x_test: (400, 700, 2)
y_test (400, 1)
13/13 [==============================] - 1s 18ms/step - loss: 0.6949 - accuracy: 0.5262
Accuracy: 0.5262071490287781
Resultados guardados en: .\Archivos_profe\modelos\OGLE-30_01_2022-19_23_29-ED\resultados_IAR\resultados.txt
contadorDeZeros: 0
Voy a leer la siguiente instancia de la GAN



In [24]:
avg = np.average(lista_accuracy_por_instancia)
print(avg)

0.5262071490287781
